In [ ]:
'''This program aims to create a datasect with a features audio and sentence
 and it will upload the dataset into huggingface.
 But this program will not create a training and test dataset.'''

In [ ]:
"""If we need to create train and test split we need to create an object
for the DatasetDict and assign each seperate split into two different list."""

In [20]:
import os
import pandas as pd

In [3]:
import huggingface_hub
huggingface_hub.login()

Token is valid.
Your token has been saved to C:\Users\basil\.cache\huggingface\token
Login successful


In [21]:
lukepath= "D:\Projects\Internship NLP\Wycliff-Mala Malasar\Dataset_Luke\Audio Files"
lukenames = os.listdir(lukepath)
print(lukenames)

luketsv = pd.read_csv(r"D:\Projects\Internship NLP\Wycliff-Mala Malasar\Dataset_Luke\transcript\line_index_audios.tsv", sep='\t')

['luk0060035.wav', 'luke00101.wav', 'luke001010.wav', 'luke001011.wav', 'luke001012.wav', 'luke001013.wav', 'luke001014.wav', 'luke001015.wav', 'luke001016.wav', 'luke001017.wav', 'luke001018.wav', 'luke001019.wav', 'luke00102.wav', 'luke001020.wav', 'luke001021.wav', 'luke001022.wav', 'luke001023.wav', 'luke001024.wav', 'luke001025.wav', 'luke001026.wav', 'luke001027.wav', 'luke001028.wav', 'luke001029.wav', 'luke00103.wav', 'luke001030.wav', 'luke001031.wav', 'luke001032.wav', 'luke001033.wav', 'luke001034.wav', 'luke001035.wav', 'luke001036.wav', 'luke001037.wav', 'luke001038.wav', 'luke001039.wav', 'luke00104.wav', 'luke001040.wav', 'luke001041.wav', 'luke001042.wav', 'luke00105.wav', 'luke00106.wav', 'luke00107.wav', 'luke00108.wav', 'luke00109.wav', 'luke00111.wav', 'luke001110.wav', 'luke001111.wav', 'luke001112.wav', 'luke001113.wav', 'luke001114.wav', 'luke001115.wav', 'luke001116.wav', 'luke001117.wav', 'luke001118.wav', 'luke001119.wav', 'luke00112.wav', 'luke001120.wav', 'l

In [22]:
from datasets import load_dataset
from datasets import Dataset
from datasets import Audio

In [23]:
lukenewvers=[]
lukenewsen =[]
for i in range(0,50):
    lukenewvers.append(str(lukepath)+'\\'+str(luketsv['audio_path'][i])+'.wav')
    #lukenewvers.append(luketsv['path'][i])
    lukenewsen.append(luketsv['label'][i])


In [24]:
from datasets import Dataset, Audio, Value, Features

dataset_dict = {
    "audio_path": lukenewvers,
    "sentence": lukenewsen
}

features = Features({
    "audio_path": Audio(),
    "sentence": Value("string")
})

dataset = Dataset.from_dict(dataset_dict).cast(features)


Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
dataset.push_to_hub("basilkr/tamil_features_50audios")

  0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
from datasets import load_dataset, DatasetDict

dataset=DatasetDict()

dataset = load_dataset("basilkr/tamil_features_50audios",use_auth_token=True)

Using custom data configuration basilkr--tamil_features_50audios-e833ce6655dfffb6


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/basil/.cache/huggingface/datasets/basilkr___parquet/basilkr--tamil_features_50audios-e833ce6655dfffb6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
dataset['train'][0]

{'audio_path': {'path': 'luke00101.wav',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  1.52587891e-05, ...,
         -1.52587891e-05, -3.05175781e-05,  3.05175781e-05]),
  'sampling_rate': 44100},
 'sentence': '. அதுக்கப்பறமு ஆண்டவரு இயேசு வேற எழுபத்திரெண்டு பேரவு தெரிஞ்சுகொண்டு, தான் போக இருந்த எல்லா ஊருகளுக்குமு, அவர்களவு ரெண்டு ரெண்டு பேரா போக சென்னரு. '}

In [28]:
from datasets import Audio

In [29]:
dataset = dataset.cast_column('audio_path',Audio(sampling_rate = 16000))

In [30]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio_path', 'sentence'],
        num_rows: 50
    })
})

In [31]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


In [32]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small",  task="transcribe")


In [33]:
input_str = dataset["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 . அதுக்கப்பறமு ஆண்டவரு இயேசு வேற எழுபத்திரெண்டு பேரவு தெரிஞ்சுகொண்டு, தான் போக இருந்த எல்லா ஊருகளுக்குமு, அவர்களவு ரெண்டு ரெண்டு பேரா போக சென்னரு. 
Decoded w/ special:    <|startoftranscript|><|transcribe|><|notimestamps|>. அதுக்கப்பறமு ஆண்டவரு இயேசு வேற எழுபத்திரெண்டு பேரவு தெரிஞ்சுகொண்டு, தான் போக இருந்த எல்லா ஊருகளுக்குமு, அவர்களவு ரெண்டு ரெண்டு பேரா போக சென்னரு. <|endoftext|>
Decoded w/out special: . அதுக்கப்பறமு ஆண்டவரு இயேசு வேற எழுபத்திரெண்டு பேரவு தெரிஞ்சுகொண்டு, தான் போக இருந்த எல்லா ஊருகளுக்குமு, அவர்களவு ரெண்டு ரெண்டு பேரா போக சென்னரு. 
Are equal:             True


In [34]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small",  task="transcribe")


In [35]:
# def prepare_dataset(batch):
#     # load and resample audio data from 48 to 16kHz
#     audio = batch["audio_path"]

#     batch["input_length"] = len(batch["audio_path"])
#     from transformers import WhisperFeatureExtractor
#     feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


#     # compute log-Mel input features from input audio array 
#     batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
#     from transformers import WhisperTokenizer

#     tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", task="transcribe")

#     # encode target text to label ids 
#     batch["labels"] = tokenizer(batch["sentence"]).input_ids

#         # compute labels length **with** special tokens! -> total label length
#     batch["labels_length"] = len(batch["labels"])
#     return batch


In [36]:
# def prepare_dataset(batch):
#     # load and resample audio data from 48 to 16kHz
#     audio = batch["audio"]

#     # compute input length
#     batch["input_length"] = len(batch["audio"])

#     # compute log-Mel input features from input audio array 
#     batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

#     # encode target text to label ids 
#     batch["labels"] = tokenizer(batch["sentence"]).input_ids

#     # compute labels length **with** special tokens! -> total label length
#     batch["labels_length"] = len(batch["labels"])
#     return batch

In [37]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio_path"]

    # compute input length
    batch["input_length"] = len(batch["audio_path"])

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    # compute labels length
    batch["labels_length"] = len(tokenizer(batch["sentence"], add_special_tokens=False).input_ids)
    return batch

MAX_DURATION_IN_SECONDS = 30.0
max_input_length = MAX_DURATION_IN_SECONDS * 16000

def filter_inputs(input_length):
    if 0<input_length['input_length']<max_input_length:
        return input_length
    
    # """Filter inputs with zero input length or longer than 30s"""
    # return 0 < input_length < max_input_length

def filter_labels(labels_length):
    if 0 < labels_length['labels_length']:
        return labels_length

    # """Filter empty label sequences"""
    # return 0 < len(labels_length)

In [38]:
dataset = dataset.map(prepare_dataset, remove_columns= dataset.column_names["train"])

#dataset = dataset.map(filter_inputs)

#dataset = dataset.filter(filter_labels, input_columns=["labels_length"], remove_columns=dataset.column_names["labels_length"])

  0%|          | 0/50 [00:00<?, ?ex/s]

In [40]:
dataset = dataset.map(filter_labels,remove_columns=["labels_length"])

  0%|          | 0/50 [00:00<?, ?ex/s]

In [19]:
(dataset['train']['input_length'])

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3]

In [16]:
(dataset['train']['labels_length'])

[126,
 238,
 87,
 132,
 121,
 144,
 85,
 53,
 57,
 190,
 290,
 240,
 378,
 148,
 126,
 224,
 228,
 72,
 222,
 185,
 375,
 368,
 126,
 202,
 241,
 124,
 248,
 122,
 109,
 286,
 104,
 170,
 133,
 170,
 196,
 205,
 184,
 149,
 124,
 284,
 83,
 212,
 298,
 181,
 52,
 231,
 162,
 170,
 191,
 177,
 128,
 103,
 116,
 96,
 271,
 207,
 120,
 167,
 206,
 124,
 188,
 161,
 122,
 223,
 157,
 297,
 159,
 251,
 167,
 138,
 365,
 374,
 332,
 262,
 267,
 463,
 172,
 257,
 123,
 133,
 148,
 130,
 205,
 450,
 273,
 355,
 149,
 283,
 184,
 177]

In [49]:
dataset = dataset.map(filter_inputs, input_columns=["input_length"], remove_columns=["input_length"])

dataset = dataset.filter(filter_labels, input_columns=["labels_length"], remove_columns=["labels_length"])

  0%|          | 0/90 [00:00<?, ?ex/s]

TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'bool'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [35]:
MAX_DURATION_IN_SECONDS = 30.0
max_input_length = MAX_DURATION_IN_SECONDS * 16000

def filter_inputs(input_length):
    """Filter inputs with zero input length or longer than 30s"""
    return 0 < input_length < max_input_length

def filter_labels(labels_length):
    """Filter empty label sequences"""
    return 0 < len(labels_length)

In [36]:
dataset = dataset.filter(filter_inputs, input_columns=["input_length"], remove_columns=["input_length"])

TypeError: DatasetDict.filter() got an unexpected keyword argument 'remove_columns'

In [41]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)


KeyError: 'audio_path'

In [42]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [43]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [44]:
import evaluate

metric = evaluate.load("wer")


In [45]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [46]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [47]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.max_length = 500


In [48]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=40,
    gradient_checkpointing=True,
    fp16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=10,
    eval_steps=10,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)


In [49]:
from transformers import Seq2SeqTrainer
import os
os.environ["HUGGINGFACE_HUB_NO_GIT"] = "1"
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["train"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


max_steps is given, it will override any value given in num_train_epochs


In [50]:
trainer.train()

C:\Users\basil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 40
  Number of trainable parameters = 241734912


  0%|          | 0/40 [00:00<?, ?it/s]

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
***** Running Evaluation *****
  Num examples = 50

  0%|          | 0/7 [00:00<?, ?it/s]

Saving model checkpoint to ./whisper-small-hi\checkpoint-10
Configuration saved in ./whisper-small-hi\checkpoint-10\config.json
Configuration saved in ./whisper-small-hi\checkpoint-10\generation_config.json


...

Saving model checkpoint to ./whisper-small-hi\checkpoint-40
Configuration saved in ./whisper-small-hi\checkpoint-40\config.json
Configuration saved in ./whisper-small-hi\checkpoint-40\generation_config.json


{'eval_loss': 1.1846407651901245, 'eval_wer': 100.2053388090349, 'eval_runtime': 686.7788, 'eval_samples_per_second': 0.073, 'eval_steps_per_second': 0.01, 'epoch': 10.0}


Model weights saved in ./whisper-small-hi\checkpoint-40\pytorch_model.bin
Feature extractor saved in ./whisper-small-hi\checkpoint-40\preprocessor_config.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./whisper-small-hi\checkpoint-30 (score: 96.50924024640656).


{'train_runtime': 24612.7237, 'train_samples_per_second': 0.026, 'train_steps_per_second': 0.002, 'train_loss': 1.989386796951294, 'epoch': 10.0}


TrainOutput(global_step=40, training_loss=1.989386796951294, metrics={'train_runtime': 24612.7237, 'train_samples_per_second': 0.026, 'train_steps_per_second': 0.002, 'train_loss': 1.989386796951294, 'epoch': 10.0})

In [53]:
trainer.push_to_hub("whisper_tamil_50")

Cloning https://huggingface.co/basilkr/whisper-small-hi into local empty directory.
Saving model checkpoint to ./whisper-small-hi
Configuration saved in ./whisper-small-hi\config.json
Configuration saved in ./whisper-small-hi\generation_config.json
Model weights saved in ./whisper-small-hi\pytorch_model.bin
Feature extractor saved in ./whisper-small-hi\preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 1.00/922M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.50k [00:00<?, ?B/s]

To https://huggingface.co/basilkr/whisper-small-hi
   ddecc65..8564ce4  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Automatic Speech Recognition', 'type': 'automatic-speech-recognition'}, 'metrics': [{'name': 'Wer', 'type': 'wer', 'value': 100.2053388090349}]}
To https://huggingface.co/basilkr/whisper-small-hi
   8564ce4..42983ab  main -> main



'https://huggingface.co/basilkr/whisper-small-hi/commit/8564ce4acb40e5b8ffc3c7194a67b9e6116b56a6'

In [54]:
tokenizer.push_to_hub("whisper_tamil_50")

tokenizer config file saved in C:\Users\basil\AppData\Local\Temp\tmp3fh95pi0\tokenizer_config.json
Special tokens file saved in C:\Users\basil\AppData\Local\Temp\tmp3fh95pi0\special_tokens_map.json
added tokens file saved in C:\Users\basil\AppData\Local\Temp\tmp3fh95pi0\added_tokens.json
Uploading the following files to basilkr/whisper_tamil_50: added_tokens.json,merges.txt,normalizer.json,special_tokens_map.json,tokenizer_config.json,vocab.json


CommitInfo(commit_url='https://huggingface.co/basilkr/whisper_tamil_50/commit/38a76389e20149fbadbeda8c300d06825c5e100a', commit_message='Upload tokenizer', commit_description='', oid='38a76389e20149fbadbeda8c300d06825c5e100a', pr_url=None, pr_revision=None, pr_num=None)

In [51]:
from transformers import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="D:\Projects\Internship NLP\Wycliff-Mala Malasar\Code\whisper-small-hi\checkpoint-30", tokenizer=tokenizer, hub_model_id="basilkr/whisper_tamil_50"
)

ImportError: 
PushToHubCallback requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.
